# Note: Advanced functions like melt, pivot table won't be there for the mid-term exam. It is good to know it and it may come in the end semester and will be useful for your project.

# Pandas Questions

This notebook contains **6 diverse, scenario-based Pandas problems**.  
Each problem includes:
- Clear instructions on how to construct the DataFrame
- A hint pointing to useful Pandas functions
- A short explanation of what the key function(s) do and example usage

**Setup:** Run the import cell before attempting the questions.
---


## Setup
Run the import cell below before attempting the questions.

In [1]:
import pandas as pd

# Only pandas is imported in this notebook


## Q1 — Grouping, aggregation and filtering

**Instructions (DataFrame construction):**
Create `df_sales` with columns: `OrderID`, `CustomerID`, `Product`, `Units`, `UnitPrice`, `OrderDate`.
- Create ~12 rows across 4 customers (CustomerID 1–4) and 5 products (P1–P5).
- `OrderDate` as strings like `'2025-01-03'`.

**Task:**
1. Add `Amount = Units * UnitPrice`.
2. Compute total `Amount` per `CustomerID`.
3. Show only customers with total > 300, sorted descending.

**Hint:** Use `.groupby()` and `.sum()` after creating the `Amount` column. You can filter using boolean indexing.

**Function explanation — `groupby`:**
`groupby()` splits the DataFrame into groups based on column values. You then apply aggregation functions (e.g., `sum`, `mean`, `count`) to each group. Example:
```python
df.groupby('CustomerID')['Amount'].sum()
```
This returns total `Amount` per `CustomerID`.


In [24]:
# YOUR CODE HERE: build df_sales and solve Q1

df_sales=pd.DataFrame({
    'orderId': range(1,13),
    'customerId': [1,1,1,1,2,2,2,2,3,3,3,3],
    'productId': [1,1,2,2,3,3,1,1,2,2,3,3],
    'units': [1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2],
    'price': [10, 20, 30, 40, 50, 60, 10, 20, 30, 40, 50, 60],
    'orderDate': pd.date_range(start='2023-01-01', periods=12, freq='D')
})
df_sales['Amount']=df_sales['units'] * df_sales['price']

print(df_sales)
print("Sum by Customer ID " ,df_sales.groupby('customerId')['Amount'].sum())

#Show only customers with total > 300, sorted descending.
result = df_sales.groupby('customerId')['Amount'].sum()

print("Result of group by ", result)
result = result[result > 200].sort_values(ascending=False)
print(result)











    orderId  customerId  productId  units  price  orderDate  Amount
0         1           1          1      1     10 2023-01-01      10
1         2           1          1      2     20 2023-01-02      40
2         3           1          2      1     30 2023-01-03      30
3         4           1          2      2     40 2023-01-04      80
4         5           2          3      1     50 2023-01-05      50
5         6           2          3      2     60 2023-01-06     120
6         7           2          1      1     10 2023-01-07      10
7         8           2          1      2     20 2023-01-08      40
8         9           3          2      1     30 2023-01-09      30
9        10           3          2      2     40 2023-01-10      80
10       11           3          3      1     50 2023-01-11      50
11       12           3          3      2     60 2023-01-12     120
Sum by Customer ID  customerId
1    160
2    220
3    280
Name: Amount, dtype: int64
Result of group by  customerId


## Q2 — Join (merge) and derived boolean column

**Instructions (DataFrame construction):**
Create `df_employees` with `EmpID`, `Name`, `DeptID` (6 rows). Create `df_depts` with `DeptID`, `DeptName`, `Manager` (3 rows).

**Task:**
1. Merge into `df_full` that includes DeptName and Manager.
2. Create `IsManagedByAlice` == True when Manager == 'Alice'.
3. Show counts of employees per DeptName and number managed by Alice.

**Hint:** Use `merge()` to join the tables and boolean comparison `==` to produce the new column.

**Function explanation — `merge`:**
`merge()` combines two DataFrames on a key column (like SQL JOIN). Example:
```python
df_employees.merge(df_depts, on='DeptID', how='left')
```
This adds department details to each employee row. For booleans, compare a column to a value:
```python
df['IsManagedByAlice'] = df['Manager'] == 'Alice'
```


In [38]:
# YOUR CODE HERE: build df_employees, df_depts and solve Q2

df_employees = pd.DataFrame({
    'empId': range(1,11),
    'empName': ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank', 'Grace', 'Hannah', 'Ian', 'Jane'],
    'deptId': [1, 2, 1, 3, 2, 1, 3, 2, 1, 3]})

#print(df_employees)

df_depts = pd.DataFrame({
    'deptId': [1, 2, 3],
    'deptName': ['HR', 'Sales', 'IT'],
    'deptManager':['Alice','Bob','Charlie']
})

#print(df_depts)

df_full=df_employees.merge(df_depts, on='deptId', how='left')

df_full['IsManagedByAlice']=  (df_full['deptManager'] =='Alice')
print(df_full)

groupedByDept=df_full.groupby('deptId').size()

#print dept names having more than 3 employees
result2 = df_full.groupby('deptId').size()
result2 = result2[result2 > 3]
print("Dept with more than 3 employees ", result2)



   empId  empName  deptId deptName deptManager  IsManagedByAlice
0      1    Alice       1       HR       Alice              True
1      2      Bob       2    Sales         Bob             False
2      3  Charlie       1       HR       Alice              True
3      4    David       3       IT     Charlie             False
4      5      Eva       2    Sales         Bob             False
5      6    Frank       1       HR       Alice              True
6      7    Grace       3       IT     Charlie             False
7      8   Hannah       2    Sales         Bob             False
8      9      Ian       1       HR       Alice              True
9     10     Jane       3       IT     Charlie             False
Dept with more than 3 employees  deptId
1    4
dtype: int64


## Q3 — Reshape: melt and pivot

**Instructions (DataFrame construction):**
Create `df_quarterly` (wide) with columns `Company`, `Q1`, `Q2`, `Q3`, `Q4` for 5 companies (numeric revenue).

**Task:**
1. Use `melt` to convert to long format (`Company`, `Quarter`, `Revenue`).
2. Pivot back to wide using `pivot_table`.
3. Compute each company's annual revenue and append as column to the pivot result.

**Hint:** `pd.melt()` and `pivot_table()` are complementary — melt turns wide -> long; pivot_table (or pivot) can aggregate and reshape long -> wide.

**Function explanation — `melt` and `pivot_table`:**
- `melt(df, id_vars=[...], value_vars=[...])` turns columns into rows, useful for tidy/long-format data.
- `pivot_table(index=..., columns=..., values=..., aggfunc=...)` reshapes long data to wide and allows aggregation. Example:
```python
df_long = pd.melt(df_quarterly, id_vars=['Company'], value_vars=['Q1','Q2','Q3','Q4'], var_name='Quarter', value_name='Revenue')
df_wide = df_long.pivot_table(index='Company', columns='Quarter', values='Revenue')
```


In [ ]:
# YOUR CODE HERE: build df_quarterly and solve Q3


## Q4 — Time series: rolling mean and resample

**Instructions (DataFrame construction):**
Create `df_temp` with daily `Date` (as strings) from '2025-03-01' to '2025-03-10' and a `Temperature` float column.

**Task:**
1. Convert `Date` to datetime and set as index.
2. Add `Temp_3d_avg` as 3-day rolling mean of Temperature.
3. Resample to weekly frequency and report mean temperature per week.

**Hint:** Convert dates with `pd.to_datetime()`, set as index, then use `.rolling()` and `.resample()`.

**Function explanation — `rolling` and `resample`:**
- `rolling(window=n).mean()` computes a moving average over `n` rows (good for smoothing). Example: `df['Temperature'].rolling(window=3).mean()`.
- `resample('W')` groups data into fixed time windows (here weekly); you can then call aggregation like `.mean()` to get weekly averages.


In [ ]:
# YOUR CODE HERE: build df_temp and solve Q4


df_temp=pd.DataFrame({
    'date':pd.date_range(start='2025-03-01', periods=10, freq='D'),
    'temprature':[30,32,31,29,28,27,26,25,24,23]\
})

df_temp['date']=pd.to_datetime(df_temp['date'])

df_temp.set_index('date', inplace=True)

df_temp['temprature_3d_avg']=df_temp['temprature'].rolling(window=3).mean()

resampled_df = df_temp.resample('W').mean()

print(resampled_df)

            temprature  temprature_3d_avg
date                                     
2025-03-02   31.000000                NaN
2025-03-09   27.142857          28.142857
2025-03-16   23.000000          24.000000


## Q5 — Missing data handling and imputation

**Instructions (DataFrame construction):**
Create `df_products` with columns `SKU`, `Category`, `Price`, `Stock` (8 rows). Include some `NaN` values in `Price` and `Stock`. At least 3 categories.

**Task:**
1. Show number of missing values per column.
2. Fill missing `Price` with median `Price` of the same `Category` and missing `Stock` with 0.
3. After imputation, show SKUs where `Stock == 0` or `Price > 100`.

**Hint:** Use `isna()`, `fillna()` and `groupby().transform()` to fill per-group values.

**Function explanation — `isna`, `fillna`, and `transform`:**
- `isna()` identifies missing values. `df.isna().sum()` gives counts.
- `fillna(value)` replaces missing values with `value` (or use `inplace=True` to modify the DataFrame).
- `groupby(...).transform()` applies a function per group and returns a Series aligned with the original DataFrame, useful for filling missing values using group statistics, e.g.:
```python
df['Price'] = df.groupby('Category')['Price'].transform(lambda x: x.fillna(x.median()))
```


In [61]:
# YOUR CODE HERE: build df_products and solve Q5



df_products=pd.DataFrame({'SKU':['A1','A2','A3','A4','A5'],
                          'Category':['Electronics','Electronics','Furniture','Furniture','Clothing'],
                          'Price':[100,200,300,float('NaN'),float('NaN')],
                          'Stock':[50,60,70,float('NaN'),float('NaN')]})


df_products.isna().sum()

df_products.fillna({'Price':df_products['Price'].mean(), 'Stock':0}, inplace=True)



df_products['Price'] = df_products.groupby('Category')['Price'].transform(lambda x: x.fillna(x.median()))
print(df_products)

  SKU     Category  Price  Stock
0  A1  Electronics  100.0   50.0
1  A2  Electronics  200.0   60.0
2  A3    Furniture  300.0   70.0
3  A4    Furniture  200.0    0.0
4  A5     Clothing  200.0    0.0


## Q6 — Apply/custom logic and string operations

**Instructions (DataFrame construction):**
Create `df_courses` with columns `CourseID`, `Title`, `Enrolled`, `StartDate` for 6 courses. Some titles should contain 'Intro' or 'Advanced'. `StartDate` as strings.

**Task:**
1. Create `Level`: 'Beginner' if Title contains 'Intro', 'Advanced' if contains 'Advanced', else 'Intermediate'. Use `apply` or vectorized string ops.
2. Create `StartMonth` derived from `StartDate`.
3. Show average `Enrolled` per `Level`.

**Hint:** Use `.str.contains()` for text checks and either `.apply()` or `np.where` for conditional column creation. Convert `StartDate` to datetime to extract month.

**Function explanation — `str.contains`, `apply`, and datetime accessor `.dt`:**
- `df['Title'].str.contains('Intro')` returns a boolean Series marking rows where 'Intro' appears.
- `apply()` runs a function row- or column-wise; useful for custom logic but can be slower than vectorized ops.
- After converting `StartDate` to datetime (`pd.to_datetime()`), use `.dt.month` to get the month number. Example:
```python
df['StartDate'] = pd.to_datetime(df['StartDate'])
df['StartMonth'] = df['StartDate'].dt.month
```


In [ ]:
# YOUR CODE HERE: build df_courses and solve Q6
df_courses=pd.DataFrame({
    'courseId':[1,2,3,4,5,6],
    'Title':['Intro Python','Advanced Python','Intro ML','Advanced ML','Data Science','AI'],
    "enrolled":[30,25,20,15,10,5],
    'startDate':pd.to_datetime(['2023-01-10','2023-02-15','2023-03-20','2023-04-25','2023-05-30','2023-06-05'])
})

df_courses['Level']= df_courses['Title'].apply(lambda x: level_from_title(x))


df_courses['startDate']=pd.to_datetime(df_courses['startDate'])

df_courses['startMonth']=df_courses['startDate'].dt.month

print(df_courses.sort_values(by='Level'))

avg=df_courses.groupby('Level')['enrolled'].mean()




def level_from_title(title):
    if 'Intro' in title :
        return 'Beginner'
    elif 'Advanced' in title:
        return 'Advanced'
    else:
        return 'Intermediate'

   courseId            Title  enrolled  startDate         Level  startMonth
1         2  Advanced Python        25 2023-02-15      Advanced           2
3         4      Advanced ML        15 2023-04-25      Advanced           4
0         1     Intro Python        30 2023-01-10      Beginner           1
2         3         Intro ML        20 2023-03-20      Beginner           3
4         5     Data Science        10 2023-05-30  Intermediate           5
5         6               AI         5 2023-06-05  Intermediate           6
Level
Advanced        20.0
Beginner        25.0
Intermediate     7.5
Name: enrolled, dtype: float64


: 